<hr style="border:2px solid coral"></hr>

# Practice

<hr style="border:2px solid coral"></hr>

Fix the following code so that it handles the ghost cell at the end points of the domain correctly by comparing the source or destination ranks to `MPI_PROC_NULL`.  

### Tips

* You will know that if a source on the left is MPI_PROC_NULL, than you are at a left boundary.  Same for the right boundary. 

* Do not compare to rank 0 or rank `nprocs-1`, since you won't in general know that rank 0 is the left boundary or that `nprocs-1` is the right boundary.



In [4]:
%%file cart_practice_01.c

#include <mpi.h>
#include <math.h>

#include <stdio.h>
#include <stdlib.h>

void main(int argc, char** argv)
{
    MPI_Init(&argc, &argv);

    int rank, nprocs;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &nprocs);
    
    int root = 0;
    
    double qmem[3];    
    double *q = &qmem[1];

    double h = 1.0/nprocs;  
    
    // # Fill in cell centered mesh locations. 
    q[0] = (rank+0.5)*h;
    
    double arr[nprocs];
    MPI_Gather(q,1,MPI_DOUBLE,arr,1,MPI_DOUBLE,root,MPI_COMM_WORLD);
    if (rank == root)
    {
        printf("f(x) = x (cell centers) : \n");
        printf("%8s : ","Rank");
        for(int p = 0; p < nprocs; p++)
            printf("%10d",p);
        printf("\n");
        
        printf("%8s : ","f(x)");
        for(int p = 0; p < nprocs; p++)
            printf("%10.4f",arr[p]);
        printf("\n");
    }        

    // # Fill in q[-1] and q[1] with data from neighbors */
    MPI_Comm comm_cart;
    int ndim = 1;
    int dims[1] = {nprocs};
    int periodicity[1] = {0};
    int reorder = 0;
    MPI_Cart_create(MPI_COMM_WORLD, ndim, dims,  periodicity, reorder, &comm_cart);

    // #  Fill ghost value q[-1]  (shift values right)
    int dir = 0;
    int disp = 1;
    int source, dest;
    int ierr = MPI_Cart_shift(comm_cart, dir, disp, &source, &dest);

    int tag = 0;
    MPI_Sendrecv(&q[0], 1, MPI_DOUBLE, dest, tag, 
                 &q[-1], 1, MPI_DOUBLE, source, tag, 
                 comm_cart, MPI_STATUS_IGNORE);

    // #  Fill ghost value q[1]
    disp = -1;  
    MPI_Cart_shift(comm_cart, dir, disp, &source, &dest);

    tag = 1;
    ierr = MPI_Sendrecv(&q[0], 1, MPI_DOUBLE, dest, tag, 
                        &q[1], 1, MPI_DOUBLE, source, tag, 
                        comm_cart, MPI_STATUS_IGNORE);

    // # Compute a derivative using left and right values
    double deriv = (q[1] - q[-1])/(2*h);
    
    MPI_Gather(&deriv,1,MPI_DOUBLE,arr,1,MPI_DOUBLE,root,MPI_COMM_WORLD);
    if (rank == root)
    {
        printf("\n");
        printf("Derivative f'(x) = 1 (cell-centers): \n");
        printf("%8s : ","Rank");
        for(int p = 0; p < nprocs; p++)
            printf("%10d",p);
        printf("\n");
        
        printf("%8s : ","f'(x)");
        for(int p = 0; p < nprocs; p++)
            printf("%10.4f",arr[p]);
        printf("\n");
    }
    

    MPI_Finalize();
}

Writing cart_practice_01.c


In [5]:
%%bash 

mpicc -o cart_practice_01 cart_practice_01.c

mpirun -n 8 cart_practice_01

f(x) = x (cell centers) : 
    Rank :          0         1         2         3         4         5         6         7
    f(x) :     0.0625    0.1875    0.3125    0.4375    0.5625    0.6875    0.8125    0.9375

Derivative f'(x) = 1 (cell-centers): 
    Rank :          0         1         2         3         4         5         6         7
   f'(x) :     0.7500    1.0000    1.0000    1.0000    1.0000    1.0000    1.0000   -3.2500
